In [1]:
# ============================================================
# CELL 1: Advanced Time Series Forecasting with Prophet + Bayesian Optimization
# ============================================================

!pip install yfinance prophet optuna statsmodels --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prophet import Prophet
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from google.colab import files

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ------------------------------------------------------------
# 1. DATA ACQUISITION & PREPROCESSING
# ------------------------------------------------------------

# 1.1 Download at least 3 years of daily data (S&P 500 index)
ticker = "^GSPC"
raw = yf.download(ticker, period="10y", interval="1d", auto_adjust=True)

# Explicitly ensure the DataFrame for Prophet has simple 'ds' and 'y' columns.
# This prevents potential MultiIndex issues from yfinance output or intermediate steps.
df_for_prophet = raw.reset_index()
df_for_prophet = df_for_prophet[['Date', 'Close']]
df_for_prophet.columns = ['ds', 'y'] # Direct column assignment for clarity and robustness

# Sort & enforce daily frequency
df_for_prophet = df_for_prophet.sort_values('ds')
df_for_prophet.set_index('ds', inplace=True)
df_for_prophet = df_for_prophet.asfreq('D')

# Interpolate missing values by time
df_for_prophet['y'] = df_for_prophet['y'].interpolate(method='time')

# Keep last 8 years to keep runtime reasonable (still >3 yrs)
if len(df_for_prophet) > 8*365:
    df_for_prophet = df_for_prophet.iloc[-8*365:]

data = df_for_prophet.reset_index() # The final 'data' DataFrame for Prophet.

# Stationarity check with ADF test (on full series)
adf_result = adfuller(data['y'])
adf_stat = adf_result[0]
adf_pvalue = adf_result[1]

print("ADF Statistic:", adf_stat)
print("ADF p-value :", adf_pvalue)

# Save cleaned dataset
data.to_csv("dataset_prophet.csv", index=False)

# 1.2 Train / Test split (last 60 days as test)
TEST_DAYS = 60
train_df = data.iloc[:-TEST_DAYS].copy()
test_df = data.iloc[-TEST_DAYS:].copy()

# Utility: metrics
def compute_metrics(y_true, y_pred):
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

short_horizon = 7
medium_horizon = 30

# ------------------------------------------------------------
# 2. BASELINE PROPHET MODEL
# ------------------------------------------------------------

baseline_model = Prophet()
baseline_model.fit(train_df)

future_base = baseline_model.make_future_dataframe(periods=TEST_DAYS)
forecast_base = baseline_model.predict(future_base)
pred_base = forecast_base[['ds', 'yhat']].tail(TEST_DAYS)

test_base = test_df.merge(pred_base, on='ds', how='left')

rmse_b_short, mae_b_short, mape_b_short = compute_metrics(
    test_base['y'].head(short_horizon),
    test_base['yhat'].head(short_horizon)
)
rmse_b_med, mae_b_med, mape_b_med = compute_metrics(
    test_base['y'].head(medium_horizon),
    test_base['yhat'].head(medium_horizon)
)

print("\nBaseline Prophet metrics:")
print("7-day  -> RMSE={:.4f}, MAE={:.4f}, MAPE={:.2f}%".format(rmse_b_short, mae_b_short, mape_b_short))
print("30-day -> RMSE={:.4f}, MAE={:.4f}, MAPE={:.2f}%".format(rmse_b_med, mae_b_med, mape_b_med))

# ------------------------------------------------------------
# 3. BAYESIAN OPTIMIZATION WITH OPTUNA
# ------------------------------------------------------------

# Split train into internal train/validation for CV
val_split_idx = int(len(train_df) * 0.85)
bo_train = train_df.iloc[:val_split_idx].copy()
bo_val = train_df.iloc[val_split_idx:].copy()

def objective(trial):
    seasonality_prior_scale = trial.suggest_float("seasonality_prior_scale", 0.1, 20.0, log=True)
    changepoint_prior_scale = trial.suggest_float("changepoint_prior_scale", 0.001, 0.5, log=True)
    n_changepoints = trial.suggest_int("n_changepoints", 5, 40)
    seasonality_mode = trial.suggest_categorical("seasonality_mode", ["additive", "multiplicative"])

    model = Prophet(
        seasonality_prior_scale=seasonality_prior_scale,
        changepoint_prior_scale=changepoint_prior_scale,
        n_changepoints=n_changepoints,
        seasonality_mode=seasonality_mode,
    )
    model.fit(bo_train)

    future = model.make_future_dataframe(periods=len(bo_val))
    forecast = model.predict(future)
    val_pred = forecast[['ds', 'yhat']].tail(len(bo_val))

    merged = bo_val.merge(val_pred, on='ds', how='left')
    rmse = sqrt(mean_squared_error(merged['y'], merged['yhat']))
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25, show_progress_bar=False)

best_params = study.best_params
best_cv_rmse = study.best_value

print("\nBest Optuna params:", best_params)
print("Best CV RMSE:", best_cv_rmse)

# ------------------------------------------------------------
# 4. FINAL OPTIMIZED PROPHET MODEL
# ------------------------------------------------------------

opt_model = Prophet(
    seasonality_prior_scale=best_params["seasonality_prior_scale"],
    changepoint_prior_scale=best_params["changepoint_prior_scale"],
    n_changepoints=best_params["n_changepoints"],
    seasonality_mode=best_params["seasonality_mode"],
)
opt_model.fit(train_df)

future_opt = opt_model.make_future_dataframe(periods=TEST_DAYS)
forecast_opt = opt_model.predict(future_opt)
pred_opt = forecast_opt[['ds', 'yhat']].tail(TEST_DAYS)

test_opt = test_df.merge(pred_opt, on='ds', how='left')

rmse_o_short, mae_o_short, mape_o_short = compute_metrics(
    test_opt['y'].head(short_horizon),
    test_opt['yhat'].head(short_horizon)
)
rmse_o_med, mae_o_med, mape_o_med = compute_metrics(
    test_opt['y'].head(medium_horizon),
    test_opt['yhat'].head(medium_horizon)
)

print("\nOptimized Prophet metrics:")
print("7-day  -> RMSE={:.4f}, MAE={:.4f}, MAPE={:.2f}%".format(rmse_o_short, mae_o_short, mape_o_short))
print("30-day -> RMSE={:.4f}, MAE={:.4f}, MAPE={:.2f}%".format(rmse_o_med, mae_o_med, mape_o_med))

# ------------------------------------------------------------
# 5. RESIDUAL ANALYSIS (OPTIMIZED MODEL)
# ------------------------------------------------------------

residuals = test_opt['y'] - test_opt['yhat']

# residuals over time
plt.figure(figsize=(10,4))
plt.plot(test_opt['ds'], residuals, marker='o')
plt.axhline(0, color='black', linestyle='--')
plt.title("Optimized Prophet Residuals (Test Set)")
plt.xlabel("Date")
plt.ylabel("Residual (y - yhat)")
plt.tight_layout()
plt.savefig("residuals_time.png")
plt.close()

# ACF & PACF plots
fig, axes = plt.subplots(1, 2, figsize=(10,4))
plot_acf(residuals, ax=axes[0], lags=30)
axes[0].set_title("Residuals ACF")
plot_pacf(residuals, ax=axes[1], lags=30, method='ywm')
axes[1].set_title("Residuals PACF")
plt.tight_layout()
plt.savefig("residuals_acf_pacf.png")
plt.close()

# ------------------------------------------------------------
# 6. SAVE ALL OUTPUT FILES
# ------------------------------------------------------------

# Baseline and optimized predictions
baseline_preds = test_base[['ds','y']].copy()
baseline_preds['yhat_baseline'] = test_base['yhat']
baseline_preds.to_csv("baseline_predictions.csv", index=False)

opt_preds = test_opt[['ds','y']].copy()
opt_preds['yhat_optimized'] = test_opt['yhat']
opt_preds.to_csv("optimized_predictions.csv", index=False)

# Metrics table
metrics_summary = pd.DataFrame({
    "Model": ["Baseline_Prophet","Baseline_Prophet","Optimized_Prophet","Optimized_Prophet"],
    "Horizon": ["7-day","30-day","7-day","30-day"],
    "RMSE": [rmse_b_short, rmse_b_med, rmse_o_short, rmse_o_med],
    "MAE":  [mae_b_short, mae_b_med, mae_o_short, mae_o_med],
    "MAPE": [mape_b_short, mape_b_med, mape_o_short, mape_o_med],
})
metrics_summary.to_csv("metrics_summary.csv", index=False)

with open("metrics_summary.txt","w") as f:
    f.write("ADF Statistic: {:.4f}\nADF p-value: {:.4f}\n\n".format(adf_stat, adf_pvalue))
    f.write(metrics_summary.to_string(index=False))

# ------------------------------------------------------------
# 7. DOWNLOAD FILES FROM COLAB
# ------------------------------------------------------------

for fname in [
    "dataset_prophet.csv",
    "baseline_predictions.csv",
    "optimized_predictions.csv",
    "metrics_summary.csv",
    "metrics_summary.txt",
    "residuals_time.png",
    "residuals_acf_pacf.png",
]:
    files.download(fname)

print("\nAll main outputs generated & downloaded.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 20.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 455, in run
    installed = install_given_reqs(
                ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/req/__init__.py", line 70, in install_given_reqs
    requirement.install(
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/req/req_install.py", line 851, in install
    install_wheel(
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/ope

ModuleNotFoundError: No module named 'optuna'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Report PDF generated and downloaded: Prophet_Bayesian_Optimization_Report.pdf
